In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [110]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import URL

In [ ]:
from config import postgresql_home 

загрузим данные из БД Postgres

In [5]:
with psycopg2.connect(
            host = postgresql_home['host'],
            user = postgresql_home['user'], 
            password = postgresql_home['password'],
            database = 'competition_analysis'
            ) as connection:
    sales = pd.read_sql('SELECT * FROM sales', connection)

C:\Users\Боря\AppData\Local\Temp\ipykernel_10864\3356721468.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales = pd.read_sql('SELECT * FROM sales', connection)


In [7]:
sales.head()

,Дата,Артель,Плата
0,2048-01-01,Каша из топора,10208.64
1,2048-01-01,Бабки Ёжки,21961.11
2,2048-01-01,Хоббитонские угощения,6905.58
3,2048-01-01,Дырявый котёл,4571.70
4,2048-01-01,Ёлки-палки,15124.69


transform. Сделаем "куб" Чеболь-Промысел-Царство

In [42]:
cube = pd.read_csv('cube.csv')
cube.head()

,Чеболь,Промысел,Царство,Плата
0,Дуремар,артефакторика,Лукоморье,54196.00
1,Дуремар,артефакторика,Нарния,126804.89
2,Дуремар,артефакторика,Тридесятое,110733.37
3,Дуремар,всячина,Лукоморье,74514.85
4,Дуремар,всячина,Нарния,24891.29


In [122]:
cube = cube.rename(columns={'Плата': 'Сумма_продаж'})

In [124]:
cube.head()

,Чеболь,Промысел,Царство,Сумма_продаж
0,Дуремар,артефакторика,Лукоморье,54196.00
1,Дуремар,артефакторика,Нарния,126804.89
2,Дуремар,артефакторика,Тридесятое,110733.37
3,Дуремар,всячина,Лукоморье,74514.85
4,Дуремар,всячина,Нарния,24891.29


запишем полученную таблицу в БД Postgres

In [11]:
sql_create_query = '''
    CREATE TABLE IF NOT EXISTS cube (
        Чеболь       VARCHAR(64), 
        Промысел     VARCHAR(64), 
        Царство      VARCHAR(64), 
        Сумма_продаж DECIMAL(10, 2)
    )
'''

In [13]:
with psycopg2.connect(
            host=postgresql_home['host'],
            user=postgresql_home['user'], 
            password=postgresql_home['password'],
            database='competition_analysis'
            ) as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql_create_query)

In [126]:
url_object = URL.create(
    "postgresql+psycopg2",
    username=postgresql_home['user'],
    password=postgresql_home['password'], 
    host=postgresql_home['host'],
    port=postgresql_home['port'],
    database='competition_analysis',
)

In [130]:
engine = create_engine(url_object)

In [136]:
cube.to_sql(
    name='cube', # имя таблицы
    con=engine,  # движок
    if_exists="replace", # если в таблице данные уже есть, заменяем их
    index=False # без индекса датафрейма
)

42